In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from bs4 import BeautifulSoup
import requests
import json
import csv
import time
import random

In [ ]:
url = "https://health-diet.ru/table_calorie/?utm_source=leftMenu&utm_medium=table_calorie"

In [ ]:
headers = {
    "accept": "*/*",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
}
req = requests.get(url, headers=headers)

In [ ]:
src = req.text

In [ ]:
with open("/content/drive/MyDrive/Учим Питон/Парсинг/Lesson_2_Заказ_на_фрилансе/index.html", 'w') as file:
    file.write(src)

In [ ]:
with open("/content/drive/MyDrive/Учим Питон/Парсинг/Lesson_2_Заказ_на_фрилансе/index.html") as file:
    src = file.read()

In [ ]:
soup = BeautifulSoup(src, 'lxml')

In [ ]:
all_product_hrefs = soup.find_all(class_="mzr-tc-group-item-href")

all_categories_dict = {}

for item in all_product_hrefs:
    item_text = item.text
    item_href = "https://health-diet.ru" + item.get('href')
    
    all_categories_dict[item_text] = item_href


print(all_categories_dict)

{'Баранина и дичь': 'https://health-diet.ru/base_of_food/food_24507/', 'Бобовые': 'https://health-diet.ru/base_of_food/food_24523/', 'Вода и напитки': 'https://health-diet.ru/base_of_food/food_24509/', 'Говядина и телятина': 'https://health-diet.ru/base_of_food/food_24502/', 'Грибы': 'https://health-diet.ru/base_of_food/food_24513/', 'Детское питание': 'https://health-diet.ru/base_of_food/food_24526/', 'Жиры и масла': 'https://health-diet.ru/base_of_food/food_24515/', 'Кондитерские изделия': 'https://health-diet.ru/base_of_food/food_24525/', 'Крупы, мука, макароны': 'https://health-diet.ru/base_of_food/food_24522/', 'Молочные продукты': 'https://health-diet.ru/base_of_food/food_24519/', 'Напитки алкогольные': 'https://health-diet.ru/base_of_food/food_24508/', 'Овощи и зелень': 'https://health-diet.ru/base_of_food/food_24512/', 'Орехи и семена': 'https://health-diet.ru/base_of_food/food_24517/', 'Полуфабрикаты': 'https://health-diet.ru/base_of_meals/meals_21253/', 'Птица': 'https://heal

In [ ]:
with open(r'/content/drive/MyDrive/Учим Питон/Парсинг/Lesson_2_Заказ_на_фрилансе/all_categories_dict.json', "w") as file:
    json.dump(all_categories_dict, file, indent=4, ensure_ascii=False)

In [ ]:
with open(r'/content/drive/MyDrive/Учим Питон/Парсинг/Lesson_2_Заказ_на_фрилансе/all_categories_dict.json') as file:
    all_categories = json.load(file)

In [ ]:
iteration_count = int(len(all_categories)) - 1
count = 0
print(f"Всего итераций: {iteration_count}")

for category_name, category_href in all_categories.items():
   
    rep = [",", " ", "-", "'"]
    for item in rep:
        if item in category_name:
            category_name = category_name.replace(item, "_")
    req = requests.get(url=category_href, headers=headers)
    src = req.text

    with open(f"/content/drive/MyDrive/Учим Питон/Парсинг/Lesson_2_Заказ_на_фрилансе/data/{count}_{category_name}.html", "w") as file:
        file.write(src)
    
    with open(f"/content/drive/MyDrive/Учим Питон/Парсинг/Lesson_2_Заказ_на_фрилансе/data/{count}_{category_name}.html") as file:
        src = file.read()
    
    soup = BeautifulSoup(src, "lxml")

    #Проверка страницы на наличие таблицы с продуктами
    alert_block = soup.find(class_="uk-alert-danger")
    if alert_block is not None:
        continue
    
    #  собираем заголовки таблицы
    table_head = soup.find(class_="mzr-tc-group-table").find("tr").find_all("th")
    product = table_head[0].text
    calories = table_head[1].text
    proteins = table_head[2].text
    fats = table_head[3].text
    carbonhydrates = table_head[4].text

    with open(f"/content/drive/MyDrive/Учим Питон/Парсинг/Lesson_2_Заказ_на_фрилансе/data/{count}_{category_name}.csv", "w", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(
            (product,
             calories,
             proteins,
             fats,
             carbonhydrates
              )
            )
    # Собираем данные продуктов
    product_data = soup.find(class_="mzr-tc-group-table").find("tbody").find_all("tr")

    product_info = [] 
    for item in product_data:
        product_tds = item.find_all("td")

        title = product_tds[0].find("a").text
        calories = product_tds[1].text
        proteins = product_tds[2].text
        fats = product_tds[3].text
        carbonhydrates = product_tds[4].text

        product_info.append(
            {
                "Title": title,
                "calories": calories,
                "proteins": proteins,
                "fats": fats,
                "carbonhydrates": carbonhydrates
            }
        )


        with open(f"/content/drive/MyDrive/Учим Питон/Парсинг/Lesson_2_Заказ_на_фрилансе/data/{count}_{category_name}.csv", "a", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(
                (title,
                 calories,
                 proteins,
                 fats,
                 carbonhydrates
                  )
                )
            
    with open(f'/content/drive/MyDrive/Учим Питон/Парсинг/Lesson_2_Заказ_на_фрилансе/data/{count}_{category_name}.json',\
              "a", encoding="utf-8") as file:
        json.dump(product_info, file, indent=4, ensure_ascii=False)
    with open(r'/content/drive/MyDrive/Учим Питон/Парсинг/Lesson_2_Заказ_на_фрилансе/data/all_product_info.json',\
              "a", encoding="utf-8") as file:
        json.dump(product_info, file, indent=4, ensure_ascii=False)
    
    count += 1
    print(f"# Итерация {count}, {category_name} записан")
    iteration_count = iteration_count - 1
    if iteration_count == 0:
        print("Работа завершена")
        break
    print(f"Осталось итераций: {iteration_count}")
    time.sleep(random.randrange(2,4))


Всего итераций: 53
# Итерация 1, Баранина_и_дичь записан
Осталось итераций: 52
# Итерация 2, Бобовые записан
Осталось итераций: 51
# Итерация 3, Вода_и_напитки записан
Осталось итераций: 50
# Итерация 4, Говядина_и_телятина записан
Осталось итераций: 49
# Итерация 5, Грибы записан
Осталось итераций: 48
# Итерация 6, Детское_питание записан
Осталось итераций: 47
# Итерация 7, Жиры_и_масла записан
Осталось итераций: 46
# Итерация 8, Кондитерские_изделия записан
Осталось итераций: 45
# Итерация 9, Крупы__мука__макароны записан
Осталось итераций: 44
# Итерация 10, Молочные_продукты записан
Осталось итераций: 43
# Итерация 11, Напитки_алкогольные записан
Осталось итераций: 42
# Итерация 12, Овощи_и_зелень записан
Осталось итераций: 41
# Итерация 13, Орехи_и_семена записан
Осталось итераций: 40
# Итерация 14, Полуфабрикаты записан
Осталось итераций: 39
# Итерация 15, Птица записан
Осталось итераций: 38
# Итерация 16, Ресторанная_еда записан
Осталось итераций: 37
# Итерация 17, Рыба_и_морепро